In [1]:
from itertools import cycle

import pandas as pd
import numpy as np


from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k


from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
test_files = pd.read_json('./meta/test.json')
train_files = pd.read_json('./meta/train.json')

In [3]:
img_width, img_height = 256, 256
batch_size = 100
epochs = 50000
no_labels = len(train_files.columns)

In [16]:
class DataGenerator:
    
    def read_image(image_path, width, height):
        img = Image.open(image_path)
        img = img.resize((width, height))
        img = np.asarray(img)
        return img
    
    def __init__(self, files_df,path='.', batch_size=10):
        self.path = path
        self.files_df = files_df
        self.cycles = []
        for j in range(files_df.shape[0]//batch_size):
             self.cycles.append((j*batch_size, (j*batch_size)+(batch_size-1)))
        self.cycles = cycle(self.cycles)
        
    def next_batch(self):
        indexes = next(self.cycles)
        X = []
        Y = []
        for i in range(indexes[0], indexes[1]):
            row = self.files_df.loc[i]
            for j in range(len(row)):
                X.append(DataGenerator.read_image(self.path + "/" + row[j] + ".jpg", img_width, img_height))
                y = np.zeros(101)
                y[j] = 1
                Y.append(y)
        return (np.asarray(X), np.asarray(Y))

In [17]:
dg = DataGenerator(train_files, path="./images", batch_size=5)

In [18]:
X_train, Y_train = dg.next_batch()


In [11]:

base_model = applications.Xception(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
for layer in base_model.layers[:25]:
    layer.trainable = False
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 127, 127, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 127, 127, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 127, 127, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [12]:
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(no_labels, activation="softmax")(x)
model = Model(input = base_model.input, output = predictions)
model.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])


/home/isitha/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  import sys


In [20]:
model.fit(x=X_train, y=Y_train)

Epoch 1/1


ResourceExhaustedError: OOM when allocating tensor with shape[131072,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training/SGD/Variable_126/Assign = Assign[T=DT_FLOAT, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training/SGD/Variable_126, training/SGD/zeros_126)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training/SGD/Variable_126/Assign', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/isitha/.local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/isitha/.local/lib/python3.6/site-packages/traitlets/config/application.py", line 

(404, 256, 256, 3)